<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/ipywidgets_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

# geemap and ipywidgets Demo

Author: Qiusheng Wu ([Website](https://wetlands.io) - [GitHub](https://github.com/giswqs))

**Keyboard shortcuts for Jupyter notebook:**

- **Shift-Enter**: run cell, select below
- **Ctrl-Enter**: run selected cells
- **Alt-Enter**: run cell and insert below
- **Ctrl-/**: comment
- **Tab**: code completion or indent
- **Shift-Tab**: tooltip

## Install geemap

In [ ]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

## Import libraries

In [ ]:
import ee
import geemap
import ipyleaflet
import ipywidgets as widgets

If you are using a VPN to access GEE, you might need to uncomment the following line and change the port to your Internet proxy.

In [ ]:
# geemap.set_proxy(port=1080)

## Create an interactive map

In [ ]:
Map = geemap.Map(center=[37.71, 105.47], zoom=4)
Map

## Add Earth Engine data

### Add raster data

In [ ]:
dem = ee.Image('USGS/SRTMGL1_003')

vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, vis_params, 'DEM')

### Add vector data

In [ ]:
fc = ee.FeatureCollection('users/giswqs/public/chn_admin_level2')
Map.addLayer(fc, {}, 'China admin level2')

## Change layer opacity

In [ ]:
Map

In [ ]:
dem_layer = Map.find_layer('DEM')
dem_layer.interact(opacity=(0, 1, 0.1))

In [ ]:
vector_layer = Map.find_layer('China admin level2')
vector_layer.interact(opacity=(0, 1, 0.1))

## Widget list

https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

### Numeric widgets

#### IntSlider

In [ ]:
int_slider = widgets.IntSlider(
    value=2000, min=1984, max=2020, step=1, description='Year:'
)
int_slider

In [ ]:
int_slider.value

#### FloatSlider

In [ ]:
float_slider = widgets.FloatSlider(
    value=0, min=-1, max=1, step=0.05, description='Threshold:'
)
float_slider

In [ ]:
float_slider.value

#### IntProgress

In [ ]:
int_progress = widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='',  # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
)
int_progress

In [ ]:
int_text = widgets.IntText(
    value=7,
    description='Any:',
)
int_text

In [ ]:
float_text = widgets.FloatText(
    value=7.5,
    description='Any:',
)
float_text

### Boolean widgets

#### ToggleButton

In [ ]:
toggle_button = widgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
)
toggle_button

In [ ]:
toggle_button.value

#### Checkbox

In [ ]:
checkbox = widgets.Checkbox(
    value=False, description='Check me', disabled=False, indent=False
)
checkbox

In [ ]:
checkbox.value

### Selection widgets

#### Dropdown

In [ ]:
dropdown = widgets.Dropdown(
    options=['USA', 'China', 'India'], value='China', description='Country:'
)
dropdown

In [ ]:
dropdown.value

#### RadioButtons

In [ ]:
radio_buttons = widgets.RadioButtons(
    options=['USA', 'China', 'India'], value='China', description='Country:'
)
radio_buttons

In [ ]:
radio_buttons.value

### String widgets

#### Text

In [ ]:
text = widgets.Text(
    value='USA',
    placeholder='Enter a country name',
    description='Country:',
    disabled=False,
)
text

In [ ]:
text.value

#### Textarea

In [ ]:
widgets.Textarea(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False,
)

### Button

In [ ]:
button = widgets.Button(
    description='Click me',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
)
button

### Date picker

In [ ]:
widgets.DatePicker(description='Pick a Date', disabled=False)

### Color picker

In [ ]:
widgets.ColorPicker(
    concise=False, description='Pick a color', value='blue', disabled=False
)

### Output widget

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
out

In [ ]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

In [ ]:
from IPython.display import YouTubeVideo

out.clear_output()
with out:
    display(YouTubeVideo('7qRtsTCnnSM'))
out

In [ ]:
out.clear_output()
with out:
    display(widgets.IntSlider())
out

## Add a widget to the map

In [ ]:
Map = geemap.Map(center=[37.71, 105.47], zoom=4)

dem = ee.Image('USGS/SRTMGL1_003')
fc = ee.FeatureCollection('users/giswqs/public/chn_admin_level2')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, vis_params, 'DEM')
Map.addLayer(fc, {}, 'China admin level2')

Map

In [ ]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [ ]:
with output_widget:
    print('Nice map!')

In [ ]:
output_widget.clear_output()

In [ ]:
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)

        with output_widget:
            output_widget.clear_output()

            try:
                admin1_id = selected_fc.first().get('ADM1_ZH').getInfo()
                admin2_id = selected_fc.first().get('ADM2_ZH').getInfo()
                Map.layers = Map.layers[:4]
                geom = selected_fc.geometry()
                layer_name = admin1_id + '-' + admin2_id
                Map.addLayer(
                    ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name
                )
                print(layer_name)
            except Exception as e:
                print('No feature could be found')
                Map.layers = Map.layers[:4]

        Map.default_style = {'cursor': 'pointer'}


Map.on_interaction(handle_interaction)